# NLQ to SQL using watsonx and langchain

## 1. Importing Libraries

In [ ]:
# !pip install ibm-generative-ai
# !pip install langchain
# !pip install langchain-experimental 

In [ ]:
import os
from dotenv import load_dotenv

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

## 2. Setting up the API key and URL

In [ ]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

## 3. Defining the model and parameters

In [ ]:
model_id = "meta-llama/llama-2-70b-chat" #bigcode/starcoder #meta-llama/llama-2-70b-chat

In [ ]:
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=1000,
            min_new_tokens=10,
            temperature=0.7,
        )

## 4. Creating the langchain interface

In [ ]:
llm = LangChainInterface(model=model_id, params=params, credentials=creds)

## 5. Loading the DATABase

In [ ]:
db = SQLDatabase.from_uri("sqlite:///data/chinook.db")

## 6. Creating SQL chain

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

## 7. Test

In [ ]:
db_chain.run("how many albums are there?")

In [ ]:
db_chain.run("tell me the name of the artist of the album with id 1")

In [ ]:
db_chain.run("how many tables are there?")

In [ ]:
db_chain.run("how many columns are there in the table album?")

In [ ]:
db_chain.run("Total number of tracks in the database?")